In [1]:
import torch

In [2]:
import torch

from datasets import load_dataset
dataset = load_dataset("copenlu/answerable_tydiqa")
train_set = dataset["train"]
validation_set = dataset["validation"]

Using custom data configuration copenlu--nlp_course_tydiqa-9ffd3d37cf2899c6
Reusing dataset parquet (/home/lyk/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-9ffd3d37cf2899c6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
dataset

DatasetDict({
    validation: Dataset({
        features: ['question_text', 'document_title', 'language', 'annotations', 'document_plaintext', 'document_url'],
        num_rows: 13325
    })
    train: Dataset({
        features: ['question_text', 'document_title', 'language', 'annotations', 'document_plaintext', 'document_url'],
        num_rows: 116067
    })
})

In [4]:
validation_set

Dataset({
    features: ['question_text', 'document_title', 'language', 'annotations', 'document_plaintext', 'document_url'],
    num_rows: 13325
})

In [8]:
train_set[0]

{'question_text': 'Milloin Charles Fort syntyi?',
 'document_title': 'Charles Fort',
 'language': 'finnish',
 'annotations': {'answer_start': [18],
  'answer_text': ['6. elokuuta (joidenkin lähteiden mukaan 9.) 1874']},
 'document_plaintext': 'Charles Hoy Fort (6. elokuuta (joidenkin lähteiden mukaan 9.) 1874 – 3. toukokuuta 1932) oli yhdysvaltalainen kirjailija ja paranormaalien ilmiöiden tutkija.',
 'document_url': 'https://fi.wikipedia.org/wiki/Charles%20Fort'}

In [10]:
finnishSet = train_set.filter(lambda x: x['language'] == "finnish")

Loading cached processed dataset at /home/lyk/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-9ffd3d37cf2899c6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-4ef3ba659b2e9493.arrow


In [12]:
japaneseSet = train_set.filter(lambda x: x['language'] == "japanese")

  0%|          | 0/117 [00:00<?, ?ba/s]

In [13]:
japaneseSet[0]

{'question_text': '“ダン” ダニエル・ジャドソン・キャラハンの出身はどこ',
 'document_title': 'ダニエル・J・キャラハン',
 'language': 'japanese',
 'annotations': {'answer_start': [35], 'answer_text': ['カリフォルニア州サンフランシスコ']},
 'document_plaintext': '“ダン”こと、ダニエル・ジャドソン・キャラハンは1890年7月26日、カリフォルニア州サンフランシスコに商社員チャールズ・ウィリアム・キャラハンとローズ・ウィーラー・キャラハンの子として生まれる[2][3]。弟は、のちに中将に昇進し戦艦ミズーリ (USS Missouri, BB-63) 艦長時代には、艦に突入した特攻隊員に礼を尽くした（アナポリス1918年組）[4]である。キャラハン家の信仰はローマ・カトリックであった[2][3]。キャラハンは海軍兵学校（アナポリス）に進む前にに入学して1907年に卒業したが、のちに弟ウィリアムも同じコースをたどった[1]。その後はアナポリスに進み、1911年に卒業。卒業年次から「アナポリス1911年組」と呼称されたこの世代からは沖縄戦で戦艦大和と対決し損なったモートン・デヨ、潜水艦部隊を率いたロバート・H・イングリッシュ[5]、室蘭艦砲射撃を行った[6]、空母任務群を率いたジョン・W・リーヴス[7]らがおり、のちに第三次ソロモン海戦でともに戦うノーマン・スコットも同期である[注釈 1]。卒業後は少尉候補生として装甲巡洋艦カリフォルニア (USS California, ACR-6) に配属され、8インチ砲の担当となる。1921年5月21日付で少尉に任官した[8]。',
 'document_url': 'https://ja.wikipedia.org/wiki/%E3%83%80%E3%83%8B%E3%82%A8%E3%83%AB%E3%83%BBJ%E3%83%BB%E3%82%AD%E3%83%A3%E3%83%A9%E3%83%8F%E3%83%B3'}

In [15]:
englishSet = train_set.filter(lambda x: x['language'] == "english")

  0%|          | 0/117 [00:00<?, ?ba/s]

In [16]:
englishSet[0]

{'question_text': 'When was quantum field theory developed?',
 'document_title': 'Quantum field theory',
 'language': 'english',
 'annotations': {'answer_start': [159], 'answer_text': ['1920s']},
 'document_plaintext': 'Quantum field theory naturally began with the study of electromagnetic interactions, as the electromagnetic field was the only known classical field as of the 1920s.[8]:1',
 'document_url': 'https://en.wikipedia.org/wiki/Quantum%20field%20theory'}

In [14]:
import torch
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab
from torchtext.utils import download_from_url, extract_archive
import io


In [20]:
en_tokenizer = get_tokenizer('basic_english', language="en")
counter = Counter()
counter.update(en_tokenizer(englishSet[0]['document_plaintext']))

In [21]:
counter

Counter({'quantum': 1,
         'field': 3,
         'theory': 1,
         'naturally': 1,
         'began': 1,
         'with': 1,
         'the': 4,
         'study': 1,
         'of': 2,
         'electromagnetic': 2,
         'interactions': 1,
         ',': 1,
         'as': 2,
         'was': 1,
         'only': 1,
         'known': 1,
         'classical': 1,
         '1920s': 1,
         '.': 1,
         '[8]': 1,
         '1': 1})

In [23]:
def build_vocab(dataSet, tokenizer):
  counter = Counter()
  for data in dataSet:
    counter.update(tokenizer(data['document_plaintext']))
  return Vocab(counter)

en_vocab = build_vocab(englishSet, en_tokenizer)

In [26]:
en_vocab["is"]

7195

In [27]:
def data_process(dataSet):
  data = []
  for element in dataSet:
    en_tensor_ = torch.tensor([en_vocab[token] for token in en_tokenizer(element["document_plaintext"])], dtype=torch.long)
    data.append(en_tensor_)
  return data

processedEnglishData = data_process(englishSet)
print(englishSet[0]["document_plaintext"])
print(processedEnglishData[0])

Quantum field theory naturally began with the study of electromagnetic interactions, as the electromagnetic field was the only known classical field as of the 1920s.[8]:1
tensor([   17,   222,   159,    16,   377,  4714, 52657,   136, 26119,    32,
           14, 44876,  6546, 52657,    32,   222,  7642, 52657,   748,   922,
           63,   222,  6546, 26119, 52657,    26, 34334,   222,   824])


In [30]:
a = []
len(a)

0